#### Update content of the CPM Atlas - analysis plots for diurnal cycles of precipitation<br>
Author          : Team BETA<br>
Return Values   : png files<br>
Source data     : The data is preprocessed and provided by Petter Lind.

In [7]:
import numpy as np
import xarray as xr
from pathlib import Path
# regridding
import xesmf as xe
# plotting
import matplotlib
import matplotlib.pyplot as plt
# for clipping
import rioxarray

In [2]:
# path to datasets
path_data = "/mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/diurnal"

In [3]:
# define availability of models by region
models_avai_cpm = {
    'NW': {
        'CNRM': 'nwe-3_cnrm-arome_cnrm-cm5',
        'KNMI': 'alp-3_hclim-knmi_ec-earth',
        'ETHZ': 'reu-3_ethz-cclm_ecmwf-erai',
        'UKMO': 'reu-3_mohc-um10.1_hadgem3-gc3.1',
    },
    'SW': {
        'CMCC': 'swe-3_cclm_ec-earth',
        'IPSL': 'swe-3_ipsl-wrf_ipsl-cm6',
        'ETHZ': 'reu-3_ethz-cclm_ecmwf-erai',
        'UKMO': 'reu-3_mohc-um10.1_hadgem3-gc3.1'
    }
}

        # SE: ['ICTP', 'ETHZ', 'UKMO'],
        # C: ['GERICS', 'ETHZ', 'UKMO'],
        # CE: ['SMHI', 'ICTP', 'ETHZ', 'UKMO'],
        # N: ['SMHI', 'GERICS'],
        # AL: ['CNRM', 'CMCC', 'IPSL', 'KNMI', 'GERICS', 'ETHZ', 'SMHI', 'ICTP', 'UKMO']

In [4]:
# define geometries for clipping by regions
regions = {
  "NW": [
    {
      'type': 'Polygon',
      'coordinates': [[[-8.0, 40.4], [11.0, 40.4], [15.2, 58.6], [-12.5, 58.6], [-8.0, 40.4]]]
    }
  ],
  "SW": [
    {
      'type': 'Polygon',
      'coordinates': [[[-10, 30], [7.4, 33], [5.7, 48.9], [-15, 45.4], [-10, 30]]]
    }
  ],
  "SE": [
    {
      'type': 'Polygon',
      'coordinates': [[[12.5, 34.3], [28.5, 34.3], [29.4, 40.9], [11.5, 40.9], [12.5, 34.3]]]
    }
  ],
  "C": [
    {
      'type': 'Polygon',
      'coordinates': [[[5.0, 44.5], [18.0, 45.5], [18.0, 56.0], [1.0, 53.0], [5.0, 44.5]]]
    }
  ],
  "CE": [
    {
      'type': 'Polygon',
      'coordinates': [[[17.8, 41.5], [31.3, 41.5], [32.8, 51.6], [16.4, 51.6], [17.8, 41.5]]]
    }
  ],
  "N": [
    {
      'type': 'Polygon',
      'coordinates': [[[1, 50.7], [26.7, 49.7], [44.1, 70.6], [-9.4, 72.6], [1, 50.7]]]
    }
  ],
  "AL": [
    {
      'type': 'Polygon',
      'coordinates': [[[1, 40], [17, 40], [17, 50], [1, 50], [1, 40]]]
    }
  ]
}

In [ ]:
# define grid for regridding
lon = np.arange(-180, 180.1, 0.5)
lat = np.arange(-90, 90.1, 0.5)
target_grid = xr.Dataset(
    {"var":(("lat", "lon"),np.ones((len(lat), len(lon)), dtype=float))},
    coords={"lat": lat, "lon": lon}
    )
# for clipping package rioxarray to identify the coordinate system
target_grid.coords["lat"].attrs["axis"] = "Y"
target_grid.coords["lon"].attrs["axis"] = "X"

def region_clip(input_data_path, region, target_grid):
    #############################################################
    ###   regridding from model native grid to target grid    ###
    #############################################################    
    # load the input data
    input_data = xr.open_dataset(input_data_path)
    # mask input data
    # note that the mask is the same for all seasons
    input_data['mask'] = xr.where(~np.isnan(input_data["pr"].sel(hour= 0)), 1, 0)
    # create regridder
    regridder = xe.Regridder(input_data, target_grid, "bilinear")
    # use the regridder to regrid data
    input_data_regrid = regridder(input_data["pr"])
    #############################################################
    ###                   region clipping                     ###
    #############################################################
    # change filled values to 0, to avoid errors after clipping
    # input_data[f"{variable}"] = input_data[f"{variable}"].where(input_data[f"{variable}"]<1e+10, 0)
    # set-up the coordinate system known to rio
    input_data_regrid.rio.write_crs("EPSG:4326", inplace=True)
    # clip data
    clipped_data = input_data_regrid.rio.clip(regions[region], "EPSG:4326", all_touched=True)

    return clipped_data

In [5]:
# choose experiment
experiment = "rcp85"
#projects = ["CPM", "RCM"]
projects = ["CPM"]

In [6]:
for region_alias in regions.keys():
    # create empty dict to host the data
    precipitation_diurnal_cpm = {}
    precipitation_diurnal_rcm = {}
    for project in projects:
        directory = path_data + "/" + project
        # data processing for cpm runs
        for model in models_avai_cpm[f'{region_alias}'].keys():
            model_file = models_avai_cpm[f'{region_alias}'][f'{model}']
            # get the file that contains chosen model and experiment name
            ncfile = [data_file for data_file in Path(directory).iterdir()
             if(model_file in data_file.stem and experiment in data_file.stem)]
            # data regridding and clipping based on the region
            clipped_region = region_clip(ncfile[0], region_alias, target_grid)
            # save processed data to the dict
            #precipitation_diurnal_cpm[f'{model}'] = 
        # data processing for rcm runs

    # make diagram
    outfile = (f"../static/cpm_analysis/diurnal/{region_alias}_{experiment}_JJA.png")
    # plot function
    print("Create diagrams of precipitation diurnal cycles for {region_alias}.")
        

NW
[PosixPath('/mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/diurnal/CPM/nwe-3_cnrm-arome_cnrm-cm5_rcp85_diurnal_cycle_amount_pr_1hr_mean_native_grid_2041-2050_JJA.nc')]
<xarray.Dataset>
Dimensions:  (hour: 24, y: 847, x: 667)
Coordinates:
    lon      (y, x) float64 ...
    lat      (y, x) float64 ...
  * x        (x) float32 0.0 2.5 5.0 7.5 ... 1.66e+03 1.662e+03 1.665e+03
  * y        (y) float32 0.0 2.5 5.0 7.5 ... 2.11e+03 2.112e+03 2.115e+03
  * hour     (hour) int64 0 1 2 3 4 5 6 7 8 9 ... 14 15 16 17 18 19 20 21 22 23
Data variables:
    pr       (hour, y, x) float32 ...
Attributes:
    Description:    Diurnal cycle | Amount | stat: mean | thr: None
    Analysed time:  2041-2050 | JJA
[PosixPath('/mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/diurnal/CPM/alp-3_hclim-knmi_ec-earth_rcp85_diurnal_cycle_amount_pr_1hr_mean_native_grid_2041-2050_JJA.nc')]
<xarray.Dataset>
Dimensions:  (y: 837, x: 773, hour: 24)
Coordinates:
  * y        (y) float32 2e+04 2.25e+04 2.5

KeyError: 'SE'